In [1]:
import numpy as np
import sympy as sym
import json
from pydae.urisi.urisi_builder import urisi
import pydae.build_cffi as db

## System without primary local control

In [3]:
grid = urisi('cigre_eu_lv_acdc_link.hjson')
grid.uz_jacs = False
grid.construct('cigre_eu_lv_acdc_link')
grid.compile_mkl('cigre_eu_lv_acdc_link')

-0.5235987755982988
build_v2.py, line 137  g Matrix([[-0.509426708108493*V_C01_0_i - 0.127356677027123*V_C01_0_r + 0.509426708108493*V_C01_1_i + 0.127356677027123*V_C01_1_r - 0.254713354054247*V_I01_0_i - 0.0636783385135617*V_I01_0_r + 0.254713354054247*V_I01_1_i + 0.0636783385135617*V_I01_1_r + 0.0372549019607843*V_MV0_0_i + 0.00931372549019608*V_MV0_0_r - 0.0186274509803922*V_MV0_1_i - 0.00465686274509804*V_MV0_1_r - 0.0186274509803922*V_MV0_2_i - 0.00465686274509804*V_MV0_2_r - 0.849044513514155*V_R01_0_i - 0.212261128378539*V_R01_0_r + 0.849044513514155*V_R01_1_i + 0.212261128378539*V_R01_1_r - i_vsc_MV0_a_r, -0.127356677027123*V_C01_0_i + 0.509426708108493*V_C01_0_r + 0.127356677027123*V_C01_1_i - 0.509426708108493*V_C01_1_r - 0.0636783385135617*V_I01_0_i + 0.254713354054247*V_I01_0_r + 0.0636783385135617*V_I01_1_i - 0.254713354054247*V_I01_1_r + 0.00931372549019608*V_MV0_0_i - 0.0372549019607843*V_MV0_0_r - 0.00465686274509804*V_MV0_1_i + 0.0186274509803922*V_MV0_1_r - 0.00465686

In [3]:
grid = urisi('cigre_eu_lv_acdc_link.json')
grid.construct('cigre_eu_lv_acdc_link')

#grid.verbose = True
#grid.construct('acdc_7bus')


vscs_end     = [{'bus_ac':'R10','bus_dc':'S10'},
                {'bus_ac':'C09','bus_dc':'D09'}]

for vsc in vscs_end:

    bus_ac = vsc['bus_ac']
    bus_dc = vsc['bus_dc']

    V_0_r,V_0_i = sym.symbols(f'V_{bus_ac}_0_r,V_{bus_ac}_0_i', real = True)
    V_1_r,V_1_i = sym.symbols(f'V_{bus_ac}_1_r,V_{bus_ac}_1_i', real = True)
    V_2_r,V_2_i = sym.symbols(f'V_{bus_ac}_2_r,V_{bus_ac}_2_i', real = True)
    V_3_r,V_3_i = sym.symbols(f'V_{bus_ac}_3_r,V_{bus_ac}_3_i', real = True)

    V_dc_0_r,V_dc_0_i = sym.symbols(f'V_{bus_dc}_0_r,V_{bus_dc}_0_i', real = True)
    V_dc_1_r,V_dc_1_i = sym.symbols(f'V_{bus_dc}_1_r,V_{bus_dc}_1_i', real = True)


    V_an = sym.sqrt((V_0_r-V_3_r)**2 + (V_0_i-V_3_i)**2)
    V_bn = sym.sqrt((V_1_r-V_3_r)**2 + (V_1_i-V_3_i)**2)
    V_cn = sym.sqrt((V_2_r-V_3_r)**2 + (V_2_i-V_3_i)**2)

    V_dc = sym.sqrt((V_dc_0_r-V_dc_1_r)**2 + (V_dc_0_i-V_dc_1_i)**2)

    p_vsc_a,p_vsc_b,p_vsc_c = sym.symbols(f'p_vsc_a_{bus_ac},p_vsc_b_{bus_ac},p_vsc_c_{bus_ac}',real=True)
    p_vsc_a,p_vsc_b,p_vsc_c = sym.symbols(f'p_vsc_a_{bus_ac},p_vsc_b_{bus_ac},p_vsc_c_{bus_ac}',real=True)

    K_droop,T_droop = sym.symbols(f'K_droop_{bus_ac},T_droop_{bus_ac}',real=True)

    dp_vsc_a = 1/T_droop*(-p_vsc_a - K_droop*(V_an/231 - V_dc/800))
    dp_vsc_b = 1/T_droop*(-p_vsc_b - K_droop*(V_bn/231 - V_dc/800))
    dp_vsc_c = 1/T_droop*(-p_vsc_c - K_droop*(V_cn/231 - V_dc/800))

    grid.dae['f'] += [dp_vsc_a]
    grid.dae['f'] += [dp_vsc_b]
    grid.dae['f'] += [dp_vsc_c]

    grid.dae['x'] += [p_vsc_a]
    grid.dae['x'] += [p_vsc_b]
    grid.dae['x'] += [p_vsc_c]

    grid.dae['u_ini_dict'].pop(f'p_vsc_a_{bus_ac}')
    grid.dae['u_ini_dict'].pop(f'p_vsc_b_{bus_ac}')
    grid.dae['u_ini_dict'].pop(f'p_vsc_c_{bus_ac}')

    grid.dae['u_run_dict'].pop(f'p_vsc_a_{bus_ac}')
    grid.dae['u_run_dict'].pop(f'p_vsc_b_{bus_ac}')
    grid.dae['u_run_dict'].pop(f'p_vsc_c_{bus_ac}')

    grid.dae['params_dict'].update({f'K_droop_{bus_ac}':0.0,f'T_droop_{bus_ac}':1.0})

grid.compile('cigre_eu_lv_acdc_link')